Test Plan for RAG Context on Syllabuses
1. Objective
This test aims to determine the optimal chunk size for retrieving the most relevant syllabus information when given a sample question. The system will be evaluated based on its ability to return the correct chunk as the top result or within the top 3 results. We will also record the time taken for retrieval.

2. Test Setup
2.1 Tools and Libraries:
Python: For data processing.
Vector database: FAISS, Pinecone, etc.
GPT model: To generate realistic queries based on the syllabus content (OpenAI API or similar).
Embedding model: Hugging Face’s sentence-transformers, OpenAI embeddings.
2.2 Dataset:
Use syllabuses of various lengths (e.g., 3000 words for one test case).
Text sources: PDFs, DOCX, TXT.
2.3 Test Parameters:
Chunk sizes: 50, 100, 200 words.
Number of test prompts: 20 sample queries per syllabus.
Test cases:
Analyze the system’s performance for the top 1 and top 3 returned chunks.
Record time taken for retrieval.
2.4 Metrics to Track:
Accuracy: Percentage of times the correct chunk is returned as the top result (or in top 3).
Top-1 accuracy: How often the top result matches the expected chunk.
Top-3 accuracy: How often the correct chunk is within the top 3 results.
Response time: Time taken to retrieve and rank the chunks for each prompt.
3. Test Process
Step 1: Preprocess Syllabus Data
Read syllabuses from various formats (PDF, DOCX, TXT).
Split syllabus text into chunks of size 50, 100, 200 words.
Sample Python Code:


In [ ]:
python
from docx import Document
from PyPDF2 import PdfReader

def read_docx(file_path):
    doc = Document(file_path)
    return " ".join([para.text for para in doc.paragraphs])

def read_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

def chunk_text(text, chunk_size):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

Step 2: Generate GPT Prompts
Use GPT-4 (or similar) to generate sample queries for each syllabus.
Label the expected chunk for each query manually.
Example:

In [ ]:
python
import openai

def generate_gpt_prompts(syllabus_text, num_prompts=20):
    prompt = f"Generate {num_prompts} queries based on this syllabus: {syllabus_text[:2000]}..."
    response = openai.Completion.create(engine="text-davinci-003", prompt=prompt, max_tokens=1500)
    return response["choices"][0]["text"].split("\n")

Step 3: Embedding and Vectorization
Vectorize each chunk of the syllabus and the GPT-generated queries.
Store vectors in a vector database.
Example:

In [ ]:
python
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Initialize embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

def embed_chunks(chunks):
    return model.encode(chunks)

def create_faiss_index(vectors):
    d = vectors.shape[1]  # Dimension of embeddings
    index = faiss.IndexFlatL2(d)  # Create a FAISS index
    index.add(vectors)
    return index

# Example
syllabus_chunks = chunk_text(read_pdf('syllabus.pdf'), 50)  # Example with 50-word chunks
chunk_vectors = embed_chunks(syllabus_chunks)
faiss_index = create_faiss_index(np.array(chunk_vectors))

Step 4: Query Matching
Embed each generated query and use the vector database to find the most similar chunks.
Retrieve top-1 and top-3 results for each query.
Example:

In [ ]:
python
def search_faiss(query, index, chunk_vectors, k=3):
    query_vector = model.encode([query])
    D, I = index.search(query_vector, k)  # Returns top-k indices
    return I[0]  # Indices of top k results

# Example
query = "What is the grading policy?"
top_k_results = search_faiss(query, faiss_index, chunk_vectors, k=3)

Step 5: Evaluate Results
For each prompt, check if the correct chunk is in the top-1 result or top-3 results.
Record the accuracy and retrieval time.
Example:

In [ ]:
python
import time

def evaluate(query, expected_chunk, index, chunk_vectors):
    start_time = time.time()
    top_results = search_faiss(query, index, chunk_vectors, k=3)
    end_time = time.time()
    
    top1_correct = top_results[0] == expected_chunk
    top3_correct = expected_chunk in top_results[:3]
    retrieval_time = end_time - start_time
    
    return top1_correct, top3_correct, retrieval_time

# Example Evaluation
correct_chunk_id = 5  # Assume we know the correct chunk ID for the query
results = evaluate("What is the grading policy?", correct_chunk_id, faiss_index, chunk_vectors)
print(f"Top-1 Accuracy: {results[0]}, Top-3 Accuracy: {results[1]}, Time: {results[2]}s")

4. Test Scenarios
Scenario 1: Chunk size = 50 words, syllabus length = 3000 words, 20 test prompts.
Scenario 2: Chunk size = 100 words, syllabus length = 3000 words, 20 test prompts.
Scenario 3: Chunk size = 200 words, syllabus length = 3000 words, 20 test prompts.
Each scenario will test:

Top-1 Accuracy: What percentage of times the pipeline returned the correct chunk as the top result.
Top-3 Accuracy: What percentage of times the correct chunk is within the top 3 results.
Retrieval Time: How long the retrieval takes for each chunk size.
5. Reporting and Analysis
For each scenario, compile the following results:

Top-1 Accuracy: (Correct Top-1 Predictions / Total Queries) * 100
Top-3 Accuracy: (Correct Top-3 Predictions / Total Queries) * 100
Average Retrieval Time: Average time in seconds across all queries.
Example Output:

Chunk Size	Top-1 Accuracy (%)	Top-3 Accuracy (%)	Avg. Retrieval Time (s)
50 Words	80%	95%	0.15
100 Words	85%	98%	0.12
200 Words	78%	90%	0.10
Conclusion
By running these scenarios, you will have data to decide on the most efficient chunk size based on accuracy and retrieval time. This methodology ensures that you can scientifically determine the optimal chunk size for your RAG pipeline.

'''
Sections of Pipeline, along with potential implementations
    1. Document storage only in Dynamo DB
        - Only keep documents in Dyanmo DB. Relevant document info loaded via document name search or metadata.
    2. Document storage with LLM summary. 
        - We can store the documents with a LLM summary of the documents that are created when the documents are loaded. This summary can be vectorized and searchd to see if document contains necessary info. 
        - We have the potential to get creative here, because we could treat each document as its own entity or we can create a kind of file system and summarize every page of every long document.
    3. Compute Vector database, store with documents.
        - Whenever documents are uploaded, we could trigger a process to compute and upload these vectors. These are then what is sorted.


Based on 143 MB of txt files, 100000 pages roughly 250 words a page,
5 characters per word, 1 byte per character, 20% extra for spaces and punc,
it would be about 70 GB of vector data for the rag context.


Notes from meeting with Ahir
- Bigger chunks
- Lance DB?
- Some documents vectorized, some not vectorized. 
- Assume 

Some sort of data structure for the chunks
    - Mapping
    - Metadata of where it came from.

Testing Strategy
 - We are going to take in the syllabuses, and and loop through chunk sizing to see
 which one is most optimal. We are going to do this by labeling the chunks and seeing
 which chunk sizing gets the most correct. An example would be for 128 size chucks,
 we label each chunk and then we send in testing suggestions. We pair these suggestions
 up with the optimal chunk. If they get it correct, the we know that we are good to go.
 Maybe I can ask chatGPT to make testing data for me on this. 
'''
